In [6]:
'''
    Set up some parsing functions first.
'''

import io
import matplotlib.pyplot as plt
import cv2

# Helper function for decoding state information
def decode_state(state_message):
    raise NotImplementedError
    return None

# Helper function for parsing messages
def parse_message(message):
    # For image messages
    if message[0:len("Image")] == bytes("Image", 'utf8'):
        # There may be a cleaner way to do this conversion, but this works fast enough for comfort
        return {'type':"Image", 
                'value': cv2.cvtColor(plt.imread(io.BytesIO(message[len("Image"):]), format='JPG'), cv2.COLOR_BGR2RGB)}
    elif message[0:len("State")] == bytes("State", 'utf8'):
        # If we receive state information, decode it
        return {'type':"State", 
                'value': decode_state(message[len("State"):-1])}
    else:
        # Default messages are returned as-is (with type as None and value as message)
        return {'type': None, 
                'value': message}

In [5]:
'''
    Set up the communication client and begin. Keyboard Interrupt closes.
'''

import socket
import select
import numpy as np
import time
import cv2

# Plot inline
%matplotlib inline
# Set up OpenCV Window
cv2.startWindowThread()

# TCP Socket Parameters
TCP_IP = '10.0.75.1'
TCP_PORT = 5005
# Data Buffer
BUFFER_SIZE = 1024
data = b''

# End Message Token
END_TOKEN = "<EOF>"

# Set up Socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((TCP_IP, TCP_PORT))
s.setblocking(0)

# Set Start Trial Option
s.send(bytes("PlayerPrefsStrial,Practice - Hills" + END_TOKEN, 'utf8'))

# Start Scene
s.send(bytes("Scene1" + END_TOKEN, 'utf8'))

# Request First Image Frame
# s.send(bytes("ImageRequest" + END_TOKEN, 'utf8'))

# Loop until keyboard interrupt, reading/requesting data
while True:
    try:
        # Wait for data and store in buffer
        ready = select.select([s], [], [], 0)
        if ready[0]:
            try:
                data += s.recv(BUFFER_SIZE)
            except ConnectionResetError:
                print("The connection was closed.")
                break
        
        # Check if buffer has a full message
        if(bytes(END_TOKEN, 'utf8') in data):
            # Decode the message and clear it from the buffer
            idx = data.index(bytes(END_TOKEN, 'utf8'))
            while idx != -1:
                parsed_message = parse_message(data[0:idx])
            
                # If the message was an image, display it using OpenCV (faster than matplotlib)
                if parsed_message['type'] == 'Image':
                    cv2.imshow('frame', parsed_message['value'])
                    cv2.waitKey(1)

                    # Send a request for a new frame
                    s.send(bytes("KeyUP" + END_TOKEN, 'utf8'))
                    # s.send(bytes("ImageRequest" + END_TOKEN, 'utf8'))

                data = data[idx + len(END_TOKEN):-1]
                if(data != ''):
                    try:
                        idx = str(data).index(END_TOKEN)
                    except ValueError:
                            break

    # Wait for a keyboard interrupt to stop the loop
    except KeyboardInterrupt:
        break

# Close the socket and destroy the display window
s.close()
cv2.destroyAllWindows()

# Confirm exit
print('Done')

Done
